In [ ]:
import os

from utils import common, model
from victim.blackbox import Blackbox
from datasets import get_dataset
import numpy as np
import torch

import torchattacks

device = common.device_dealer(3)
blackbox = Blackbox.from_modeldir('/mnt/ywb/results/victim/gtsrb-blackbox', device)
surrogate = model.Model('/mnt/ywb/results/adversary/gtsrb-imagenet-balanced', 'Classifier', 32, 'GTSRB', 100, device,
                        complexity=64, batch_size=128,
                        resume='/mnt/ywb/results/adversary/gtsrb-imagenet-balanced/checkpoint.1000.pth.tar', train_criterion='SCE')
imagenet = get_dataset('ImageNet64', 32, True)
indexes = np.load(os.path.join(surrogate.model_dir, 'selected_.npy'))

In [ ]:
imagenet[0]

In [ ]:
query_instance = common.QueryWrapper(imagenet, indexes)
results = common.query(blackbox, query_instance, device=device)
train_instance = common.QuerySubset(imagenet, indexes, results)

In [ ]:
sm = np.load(os.path.join(surrogate.model_dir, 'statistic_matrix_.npy'))

In [ ]:
surrogate.train(train_instance, 'ModelClass')

In [ ]:
model.train_model(surrogate.model, train_instance, surrogate.model_dir, 128, testset=surrogate.testset, device=device, criterion_train=model.soft_cross_entropy, optimizer=surrogate.optimizer)

In [ ]:
selecting_pool = set(range(len(imagenet))).difference_update(indexes)
new_indices =  np.random.choice(list(selecting_pool), 10000, replace=False)
query_instance = common.QueryWrapper(imagenet, new_indices)
results = common.query(blackbox, query_instance, device=device)
train_instance.extend(new_indices, results)

In [ ]:
import torchattacks
import numpy as np
from torch.utils.data.dataloader import DataLoader
from tqdm import tqdm
attacker = torchattacks.CW(surrogate.model, c=0.1, steps=200)
attacker.set_attack_mode('targeted')
index_list = list(range(len(imagenet)))
random_selection = np.random.choice(index_list, 1000, replace=False)
random_imagenet = common.Subset(imagenet, random_selection)

all = 0
epsilon=0.
for x_t, y_t in DataLoader(random_imagenet, 256, False):
    target = torch.ones(x_t.size(0), dtype=int)
    x_adv = attacker(x_t, target)
    epsilon += (x_adv.cpu() - x_t).reshape(x_t.size(0), 32*32*3).norm(dim=1).sum()
    y_adv = surrogate(x_adv.to(device)).argmax(1)
    all += (target == y_adv.cpu()).sum()

In [ ]:
import torch
import numpy as np
x = np.load('/mnt/ywb/results/adversary/gtsrb-imagenet-targeted/statistic_matrix_.npy')